<a href="https://colab.research.google.com/github/raphaelsap/docs/blob/main/AIMO_Raphael_Caillon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'ai-mathematical-olympiad-prize:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F73231%2F8133715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240416%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240416T134853Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D67f65ac9f63d67e5a89a6f325366c2e4fa1d41a8f5e417b3d91494bee2e6e7d904221c53f29448b49821e1dcc259f2f1f7199e94ba9387594d366106cbdbbdc882ead492058899872895b970b59ec99e9da0a483c58f53f9a36b7fc71dc01ff587d9d6f5c698577c159ca5d6fe50526079bf4fe14ce52b73d6dca485cd5a6add15897377bf9dea9ee988d7ec423b1ebff552a69e99c2537c0b96c583d9a30d679ef2d65af76bb252650000d4675d0cdb464083a96d0d77f26ce0fdfb705eff40c72cb91609bd5bf4474d98ea8df42ced12bd5f15f2889fce13fb13cb049c8f9444f3b948a22d81637fd24ee6e54fd7dc97bf14bbbd681a723db77b18ba514998,bitsandbytes-0-42-0-py3-none-any-whl:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4281572%2F7369493%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240416%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240416T134853Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3De3256077b92f61aa6323976984adf0ebb58c8e524d6ef78a8803b2584be1f4b1d21f4d15e8deb33c9b46b9b44fa06cfa6e40189859b1bc56f762d91bae1da73d2e38b2f0ccb96da79847f831afb19d66fcd96a2efbaf75481ed377de76659f8ffd056e68ed4fc6f17567417b9c89f1983490e7b67534f3bb63829ba5f7b1718bfcb2f8afa8e1503389168900707b8c96a036aa39a79f226755a50adf000c5054a54b41c0cda1cd7b8d28aee480596d2bafd52812325dfb250339dcb5f51ce461bec0d86b3f9d211ed7bc1cc4c5231ebef909e47fb91580a707a32d3fff6eb95b8da6e415aec3224ad4b1e8a7538dbe9abb1a04dff6f6e4034949463daa924af5,open-math-mistral:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4720595%2F8012825%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240416%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240416T134853Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dd67c36dab0d4783a8523d465884195abcfafa40ebc4e8f3c2c912db4fdd5eaa0d98bf0d40251b10a14baf727290b1291e0152791cce61a3897c81214e0e38694559d9f48087d0f9b5a4a5f59faa3550569ccd47076ef28dfeeaaf59927c6e3f66a689c1c4a87988599332d43a0cf08a2fe635fe9823d73ed7b1c0c29e4446db7e089dc4281d27aef8c078995a7fe51406597680ad6155bf3617f9e62c83e8cf7f90616ff69e1bced448f33f552d543a28cf28ed4cc3e853fe1d66c781d55c9eb2eeb4f4c31042ff3cbb17284412cf16437fb1b703d1b70de687e529ceadb2ac59e9dd5ed29079e4b58ea8dc65ae127e684c00cda4669c313d9e2026921a7cd7e,deepseek-math:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4728129%2F8023365%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240416%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240416T134853Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2ef90bc2d0e0c850bc7641ce21a3052219b04eadfaf45690f1f98c43413d5155487f88d0a3fd68987167e43c2b4d6360fd3dbf5f49c152af812b191e192960b91ac57285873ae6973c7412d4c91b3011aedc8ffaac8a2240357bc208e8c24ec545e75e86717ced35e1681683f82b2f518d6b88bcdf912049c5f88e10a795429d0afb70314eb14130c823b3156eeb98a4fa49ba6500fd3ebacbfd9ded13cebf2d813a9b89bdf25c96d50813db7566fa47a82853ebfdb42ab06c3b2323f61713dea622f8a231d59c3883deabeeeaf605637a6abb78e94112fd509620ed22ea5c66d38664850d00b9896531c7bab0591a688cf364c70edfd17d6ddbc318b9e60b37,mistral/pytorch/7b-instruct-v0.1-hf/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3900%2F5112%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240416%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240416T134853Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1dae5b0d59b1957530253cebe150972a01355bfd205961094cf6c674d8190aceceb38e6dd881925037a6d2abeb1e0941b3694a5a8cdbbe3321aefbe6d8a89a25d15da91ecf20c0b6bfa57dd0e75b0c11c5fe3515d5a6973246e60fd135f5d3db447b7f8cc5f4a9176a2f4cf5613519f7abcb13abe4eab6ccc17bc70eef48b077895eef17f22fc827c9970d7bafc28ee65e9d73b06477cb7bff62d9ffbf19f892b578a43d000562af8b1db5ee96e03f6c7101afb883dacd23ec5046f68cd9f3c7954004b328316aabda2e8c0a67d43472336b6e32d161dc66c33b16775db1cffe27964e36feb60bf65a74d864fadb87f3c38865d340154314b1ba05f1b9bae690,mixtral/pytorch/8x7b-instruct-v0.1-hf/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F4761%2F5994%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240416%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240416T134853Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dcf1575596c5e28fae7178a4e20308790b3f2eeed6cea9e80d548cdab4c68a56afd780b88402cfba013b260a914e74af64f085967d7e5435b35ffc46c628d67672b158365fbd390c55311a05ff71996eb3a6dd2666d98216240120f1cf7b643222585796c195d5a576ecf91d9e5e6e90107af6dbd8f4b41e8ed7db87f57b73a907ace2ce0b4a6248c306269a1a331bc2f5a40548dc87a602cc693ae96c45a5caf30f4856fbe8e72e7229b29a42137d64f0471b8743501d502a825294375c074741de5cceb18a214e85fb035594476136b57395b8042bf72eabba5652a5503fa1f6e7fd35ccac502555c96371f6f9f89ed156c463391eaf3343516280e283745e1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 119437 bytes downloaded
Downloaded and uncompressed: ai-mathematical-olympiad-prize
[==================================================] 103419797 bytes downloaded
Downloaded and uncompressed: bitsandbytes-0-42-0-py3-none-any-whl
[==================================================] 11458991910 bytes downloaded
Downloaded and uncompressed: open-math-mistral
[==================================================] 10946764350 bytes downloaded
Downloaded and uncompressed: deepseek-math
[==================================================] 11936623935 bytes downloaded
Downloaded and uncompressed: mistral/pytorch/7b-instruct-v0.1-hf/1
[====                                              ] 13915299840 bytes downloaded

In [ ]:
# credits:
# https://www.kaggle.com/code/olyatsimboy/aimo-openmath-mistral-baseline
# https://www.kaggle.com/code/aatiffraz/prompt-prediction-w-mixtral-mistral7b-gemma-llama
# https://www.kaggle.com/code/thedrcat/aimo-mixtral-baseline

# Zero-shot MMOS-DeepSeekMath-7B with self-consistency and generated code reasoning evaluation

Self-consistency is a modification of the standard greedy decoding in reasoning pipelines via sampling several diverse answers followed by aggregation, e.g., most common answer ([SC-CoT paper](https://arxiv.org/pdf/2203.11171.pdf)).

In this kernel, we will consider MMOS-DeepSeekMath-7B RL-tuned backbone; in my experiments, this model produces more consistent code reasoning and the code block execution will allow us to decrease arithmetic hallucinations.

In [ ]:
!pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoConfig,
    set_seed
)

set_seed(42)

MODEL_PATH = "/kaggle/input/deepseek-math"

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

config = AutoConfig.from_pretrained(MODEL_PATH)
config.gradient_checkpointing = True


tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
#     quantization_config=quantization_config,
    config=config
)

In [ ]:
model.dtype

In [ ]:
import pandas as pd
from tqdm import tqdm
PRIVATE = True

df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/test.csv')
df.head()

In [ ]:
if len(df) < 5:
    df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
    PRIVATE = False
df.head()

In [ ]:
import gc
device = 'cuda'

In [ ]:
def naive_parse(answer):
    out = []
    start = False
    end = False
    for l in reversed(list(answer)):
        if l in '0123456789' and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True

    out = reversed(out)
    return ''.join(out)

In [ ]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype='auto',
    device_map="auto",
)

In [ ]:
print(f"Transformers Version: {transformers.__version__}")

In [ ]:
import torch

torch.backends.cuda.enable_mem_efficient_sdp(False)

In [ ]:
import re
import sys
import subprocess


def process_output(output):
    result = output

    try:
        code = output.split('```')[1][7:]

        with open('code.py', 'w') as fout:
            fout.write(code)

        batcmd = 'timeout 7 ' + sys.executable + ' code.py'
        try:
            shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
            print(shell_output)
            code_output = round(float(eval(shell_output))) % 1000
        except:
            code_output = -1

        print('CODE RESULTS', code_output)

    except Exception as e:
        print(e)
        print('ERROR PARSING')
        code_output = -1

    try:
        result_output = re.findall(r'\\boxed\{(.*)\}', result)

        print('BOXED', result_output)
        if not len(result_output):
            result_output = naive_parse(result)
        else:
            result_output = result_output[-1]

        print('BOXED', result_output)
        if not len(result_output):
            result_output = -1

        else:
            result_output = round(float(eval(result_output))) % 1000

    except Exception as e:
        print(e)
        print('ERROR PARSING')
        result_output = -1

    return result_output, code_output

In [ ]:
import re
from collections import defaultdict


tool_instruction = " The answer should be given as a non-negative modulo 1000."
tool_instruction += '\nPlease integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}.'


n_repetitions = 5 if PRIVATE else 2

total_results = []
total_answers = []

for i in tqdm(range(len(df))):
    id_ = df['id'].loc[i]
    problem = df['problem'].loc[i]

    messages = [
        {
            "role": "user",
            "content": problem + tool_instruction
        }
    ]

    query_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False
    )

    results = []
    answers = []

    for _ in tqdm(range(n_repetitions)):
        try:
            raw_output = pipeline(
                query_prompt,
                max_new_tokens=2048,
                do_sample=True,
                temperature=0.7,
                return_full_text=False
            )
            raw_output = raw_output[0]['generated_text']

            result_output, code_output = process_output(raw_output)

            torch.cuda.empty_cache()
            gc.collect()

        except Exception as e:
            print(e)
            result_output, code_output = -1, -1

        results.append(result_output)
        answers.append(code_output)

    total_results.append(results)
    total_answers.append(answers)

In [ ]:
import numpy as np
from collections import Counter

final_answers = []

for a, b in zip(total_answers, total_results):
    a = np.array(a)
    b = np.array(b)
    a[a < 0] = b[a < 0]

    pred = Counter(a.tolist()).most_common(2)

    ans = pred[0][0] if not pred[0][0] < 0 else pred[1][0]

    final_answers.append(ans)
    print(ans)


In [ ]:
df['answer'] = final_answers

In [ ]:
df

In [ ]:
df[['id','answer']].to_csv("submission.csv", header=True, index=False)

In [ ]:
df[['id','answer']].head()

In [ ]:
if not PRIVATE:
    df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
    df['model_answer'] = final_answers
    df['match'] = df.answer == df.model_answer
    print(f'{df.match.sum()} matches in {len(df)} examples')